In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import time


In [4]:
mp_drawing = mp.solutions.drawing_utils          # mediapipe 繪圖方法
mp_drawing_styles = mp.solutions.drawing_styles  # mediapipe 繪圖樣式
mp_pose = mp.solutions.pose                      # mediapipe 姿勢偵測

In [5]:
cap = cv2.VideoCapture(0)

# 三秒後開始拍攝
time.sleep(3)

start_time = time.time()
landmarks_list = []
# 啟用姿勢偵測
with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:

    if not cap.isOpened():
        print("Cannot open camera")
        exit()
    while True:
        current_time = time.time()
        ret, img = cap.read()
        if not ret:
            print("Cannot receive frame")
            break
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img)

        # 新增身體數據
        if results.pose_landmarks:
            landmarks = np.array([[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]).flatten()
            landmarks_list.append(landmarks)


        mp_drawing.draw_landmarks(
            img,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        
        cv2.imshow('MediaPipe Pose', img)

        # 更改持續拍攝時間：目前是連續取0秒
        if current_time - start_time >= 30:
            break
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

2024-03-09 16:49:56.980 python[41370:1449640] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1709974201.391689       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
landmarks_list = np.array(landmarks_list)

In [8]:
# 定義class
label = np.full((landmarks_list.shape[0], 1), 2) # <---class的名稱，更改最後這個參數，並用阿拉伯數字命名
data = np.hstack((landmarks_list, label))
columns = [f'feature_{i}' for i in range(99)] + ['Label']
df = pd.DataFrame(data, columns=columns)

In [9]:
# 完成的資料集：一筆資料共有共有99筆數據，Label為上面指定的阿拉伯數字
df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,Label
0,0.478471,0.197134,-0.469924,0.467825,0.159518,-0.459703,0.476105,0.152335,-0.459235,0.484526,...,0.550845,2.666708,0.583100,0.734861,2.694186,0.181883,0.588646,2.754248,0.168938,2.0
1,0.423801,0.206925,-0.410518,0.416325,0.172563,-0.416543,0.424644,0.163546,-0.416120,0.432508,...,0.524323,2.664149,0.641894,0.706735,2.694082,0.042756,0.553009,2.754215,0.214901,2.0
2,0.411837,0.213122,-0.413165,0.400899,0.177967,-0.422214,0.408751,0.168426,-0.421813,0.416135,...,0.520970,2.680595,0.761327,0.705031,2.704700,-0.003637,0.552343,2.772343,0.326690,2.0
3,0.397174,0.216199,-0.401141,0.385594,0.180742,-0.413148,0.393567,0.170511,-0.412752,0.401200,...,0.521083,2.696246,0.722300,0.705977,2.716694,0.023901,0.554392,2.788825,0.287802,2.0
4,0.386872,0.219034,-0.391488,0.376187,0.182276,-0.402659,0.384397,0.171595,-0.402344,0.392110,...,0.516119,2.695033,0.703819,0.698478,2.720271,0.015586,0.550278,2.789576,0.269322,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,0.415134,0.157083,-0.396966,0.408936,0.113732,-0.379134,0.416460,0.105082,-0.378691,0.424741,...,0.529663,2.706320,0.432508,0.726197,2.732184,0.004215,0.568027,2.795383,-0.016831,2.0
445,0.404851,0.161239,-0.402464,0.399042,0.118665,-0.384359,0.406923,0.110197,-0.383917,0.414964,...,0.521525,2.707487,0.410826,0.714361,2.734921,0.054678,0.560488,2.796524,-0.033873,2.0
446,0.399476,0.162936,-0.403116,0.393906,0.121053,-0.386469,0.401559,0.112661,-0.386045,0.409184,...,0.525716,2.704631,0.463488,0.718451,2.729205,0.038029,0.563436,2.794651,0.003382,2.0
447,0.393856,0.166203,-0.386699,0.387388,0.124723,-0.377557,0.395253,0.116343,-0.377143,0.403461,...,0.527240,2.704577,0.479329,0.718476,2.729652,0.008879,0.564454,2.794909,0.027001,2.0


In [10]:
# 儲存檔案，把檔名改成想要的名字，這樣一個class就製作完成了！
df.to_csv("headRight.csv")